In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.image as image
import seaborn as sns
import pandas as pd
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from shapely.ops import unary_union


sns.set_style("whitegrid")

In [3]:
cpath = 'data/map/namdinh/NamDinh_Xa.shp'
cmnd = gpd.read_file(cpath)

## Merging some area

In [4]:
# Copy the row for "Nghĩa Thắng"
nghia_thang_row = cmnd[cmnd["diaDanh"] == "Nghĩa Thắng"].copy().iloc[0]
nghia_phuc_geom = cmnd[cmnd["diaDanh"] == "Nghĩa Phúc"].geometry.iloc[0]

# Merge the geometries
merged_geom = unary_union([nghia_thang_row.geometry, nghia_phuc_geom])

# Prepare a new row as a dictionary, skipping 'NAME_3' and 'geometry'
new_row_data = {
    column: nghia_thang_row[column]
    for column in nghia_thang_row.index
    if column not in ["diaDanh", "geometry"]
}

# Add the updated 'NAME_3' and 'geometry'
new_row_data["diaDanh"] = "Phúc Thắng"
new_row_data["geometry"] = merged_geom

# Create a new GeoDataFrame row
new_gdf_row = gpd.GeoDataFrame([new_row_data], crs=cmnd.crs)

new_gdf_row
# Concatenate the new row with the original GeoDataFrame and remove the original "Nghĩa Phúc" and "Nghĩa Thắng"
cgdf = pd.concat([cmnd, new_gdf_row], ignore_index=True)

cgdf = cgdf[cgdf["diaDanh"] != "Nghĩa Phúc"]
cgdf = cgdf[cgdf["diaDanh"] != "Nghĩa Thắng"]

In [5]:
hai_an_row = cmnd[cmnd["diaDanh"] == "Hải An"].copy().iloc[0]
hai_toan_geom = cmnd[cmnd["diaDanh"] == "Hải Toàn"].geometry.iloc[0]

# Merge the geometries
merged_geom = unary_union([hai_an_row.geometry, hai_toan_geom])

# Prepare a new row as a dictionary, skipping 'geometry'
new_row_data = {
    column: hai_an_row[column]
    for column in hai_an_row.index
    if column not in ["geometry"]
}

# Add the updated 'NAME_3' and 'geometry'
new_row_data["geometry"] = merged_geom

# Create a new GeoDataFrame row
new_gdf_row = gpd.GeoDataFrame([new_row_data], crs=cgdf.crs)
cgdf = cgdf[cgdf["diaDanh"] != "Hải An"]

# Concatenate the new row with the original GeoDataFrame and remove the original 
cgdf = pd.concat([cgdf, new_gdf_row], ignore_index=True)

cgdf = cgdf[cgdf["diaDanh"] != "Hải Toàn"]
#

In [6]:
lam_row = cmnd[cmnd["diaDanh"] == "Lâm"].copy().iloc[0]
yen_xa_geom = cmnd[cmnd["diaDanh"] == "Yên Xá"].geometry.iloc[0]

# Merge the geometries
merged_geom = unary_union([lam_row.geometry, yen_xa_geom])

# Prepare a new row as a dictionary, skipping 'geometry'
new_row_data = {
    column: lam_row[column]
    for column in lam_row.index
    if column not in ["geometry"]
}

# Add the updated  'geometry'
new_row_data["geometry"] = merged_geom

# Create a new GeoDataFrame row
new_gdf_row = gpd.GeoDataFrame([new_row_data], crs=cgdf.crs)
cgdf = cgdf[cgdf["diaDanh"] != "Lâm"]

# Concatenate the new row with the original GeoDataFrame and remove the original 
cgdf = pd.concat([cgdf, new_gdf_row], ignore_index=True)

cgdf = cgdf[cgdf["diaDanh"] != "Yên Xá"]

In [8]:
cgdf[cgdf['diaDanh'] == 'Trực Phú']

,OBJECTID,maDonViHan,danhTuChun,diaDanh,Shape_Leng,Shape_Area,MaTinh,TenTinh,MaHuyen,TenHuyen,geometry
144,130,14077,Xã,Trực Phú,15898.566951,7.415665e+06,36,Tỉnh Nam Định,363,Huyện Trực Ninh,"POLYGON ((106.20644 20.21156, 106.20658 20.211..."


In [9]:
cgdf.loc[cgdf['diaDanh'] == 'Trực Phú', ['diaDanh', 'danhTuChun']] = ['Ninh Cường', 'Thị trấn']

In [13]:
cgdf.to_file('data/map/base/commune.json', driver='GeoJSON')